In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.5.0
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.3.3
numpy 1.19.4
pandas 1.1.4
sklearn 0.23.2
tensorflow 2.5.0
tensorflow.keras 2.5.0


In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

In [3]:
from sklearn.model_selection import train_test_split

x_train_all, x_test, y_train_all, y_test = train_test_split(
    housing.data, housing.target, random_state = 7)
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_all, y_train_all, random_state = 7)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.fit_transform(x_valid)
x_test_scaled = scaler.fit_transform(x_test)

In [5]:
output_dir = 'generate_csv'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
def save_2_csv(output_dir, data, name_prefix, header=None, n_parts=10):
    path_format = os.path.join(output_dir, '{}__{:02d}.csv')
    filenames = []
    
    for file_idx, row_indices in enumerate(
        np.array_split(np.arange(len(data)), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filenames.append(part_csv)
        with open(part_csv, 'wt', encoding='utf-8') as f:
            if header:
                f.write(header+'\n')
            for row_indice in row_indices:
                f.write(','.join([repr(col) for col in data[row_indice]]))
                f.write('\n')
    
    return filenames
    

train_data = np.c_[x_train_scaled, y_train]
valid_data = np.c_[x_valid_scaled, y_valid]
test_data = np.c_[x_test_scaled, y_test]
header_cols = housing.feature_names + ['MidianHouseValue']
header_str = ','.join(header_cols)

train_filenames = save_2_csv(output_dir, train_data, 'train', header_str, n_parts=20)
valid_filenames = save_2_csv(output_dir, valid_data, 'valid', header_str, n_parts=10)
test_filenames = save_2_csv(output_dir, test_data, 'test', header_str, n_parts=10)

In [6]:
import pprint
print('train filenames')
pprint.pprint(train_filenames)
print('valid filenames')
pprint.pprint(valid_filenames)
print('test filenames')
pprint.pprint(test_filenames)

train filenames
['generate_csv/train__00.csv',
 'generate_csv/train__01.csv',
 'generate_csv/train__02.csv',
 'generate_csv/train__03.csv',
 'generate_csv/train__04.csv',
 'generate_csv/train__05.csv',
 'generate_csv/train__06.csv',
 'generate_csv/train__07.csv',
 'generate_csv/train__08.csv',
 'generate_csv/train__09.csv',
 'generate_csv/train__10.csv',
 'generate_csv/train__11.csv',
 'generate_csv/train__12.csv',
 'generate_csv/train__13.csv',
 'generate_csv/train__14.csv',
 'generate_csv/train__15.csv',
 'generate_csv/train__16.csv',
 'generate_csv/train__17.csv',
 'generate_csv/train__18.csv',
 'generate_csv/train__19.csv']
valid filenames
['generate_csv/valid__00.csv',
 'generate_csv/valid__01.csv',
 'generate_csv/valid__02.csv',
 'generate_csv/valid__03.csv',
 'generate_csv/valid__04.csv',
 'generate_csv/valid__05.csv',
 'generate_csv/valid__06.csv',
 'generate_csv/valid__07.csv',
 'generate_csv/valid__08.csv',
 'generate_csv/valid__09.csv']
test filenames
['generate_csv/test__00

In [7]:
# 1.filename -> dataset
# 2.read file -> dataset -> datasets -> merge
# 3.parse csv

filename_dataset = tf.data.Dataset.list_files(train_filenames)
for filename in filename_dataset:
    print(filename)

tf.Tensor(b'generate_csv/train__18.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train__02.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train__17.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train__12.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train__10.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train__06.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train__00.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train__11.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train__03.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train__16.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train__08.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train__15.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train__14.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train__09.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train__13.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/

In [8]:
n_readers = 5
dataset = filename_dataset.interleave(
    lambda filename: tf.data.TextLineDataset(filename).skip(1),
    cycle_length = n_readers
)
for line in dataset.take(15):
    print(line.numpy())

b'0.8095613521801492,0.34518428537112195,0.22347987936517727,-0.10669826574327357,-0.17047084206891044,0.008573079922849763,-0.841838842223145,0.739231082317489,2.349'
b'-0.34681893647975504,-0.6120806264069084,-0.2993192081256999,0.04803324388614384,0.04851167062181197,-0.08686994126075745,-0.8184322867866667,0.7641648666509977,1.807'
b'-1.0163746567322829,1.86085372901967,-0.8985310852575602,-0.053064742407918725,-0.8097313310391347,0.2399721267795869,-0.7669378648264085,0.6544562155835542,0.388'
b'-0.8326912290523859,-0.21322024649939578,0.19590484389231774,0.28256634342161385,0.30623724325012375,-0.06698661284884411,-0.1770926678271232,-0.5473521892916272,2.47'
b'-1.2510048415710915,1.3024491971491523,-1.3111452352989834,0.00781415440598808,-0.2184785467741842,0.08010877566205839,-0.7482126204772251,0.6594429724502586,1.875'
b'0.5112344948394413,1.3024491971491523,0.13587658679582906,-0.10351732524563802,-0.6084358674503553,-0.10230906308235456,-0.7528939315645234,0.534774050782708

In [9]:
# tf.io.decode_csv(str, record_defaults)

sample_str = '1,2,3,4,5'
record_deafults = [tf.constant(0, dtype=tf.int32)] * 5
parsed_fields = tf.io.decode_csv(sample_str, record_deafults)
print(parsed_fields)

[<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=int32, numpy=3>, <tf.Tensor: shape=(), dtype=int32, numpy=4>, <tf.Tensor: shape=(), dtype=int32, numpy=5>]


In [10]:
# tf.io.decode_csv(str, record_defaults)

sample_str = '1,2,3,4,5'
record_deafults = [tf.constant(0, dtype=tf.int32),
                  0,
                  np.nan,
                  'hello',
                  tf.constant([])]
parsed_fields = tf.io.decode_csv(sample_str, record_deafults)
print(parsed_fields)

[<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=float32, numpy=3.0>, <tf.Tensor: shape=(), dtype=string, numpy=b'4'>, <tf.Tensor: shape=(), dtype=float32, numpy=5.0>]
